### Feature 설명
서울기온(°C) : 서울(출발지)의 시간별 기온  
서울강수량(mm): 서울(출발지)의 시간별 강수량  
서울풍속(m/s)	: 서울(출발지)의 시간별 풍속  
서울습도(%)	: 서울(출발지)의 시간별 습도  
서울이슬점온도(°C) : 서울(출발지)의 시간별 이슬점 온도  
서울일조(hr) : 서울(출발지)의 시간별 일조량  
서울일사(MJ/m2)	: 서울(출발지)의 시간별 서울일사량  
서울시정(10m)	: 서울(출발지)의 시간별 시정(시야)  
서울지면온도(°C) : 서울(출발지)의 시간별 지면온도  
부산기온(°C) 부산강수량(mm)	부산풍속(m/s)	부산습도(%)	부산이슬점온도(°C)	부산일조(hr)	부산일사(MJ/m2)	부산시정(10m)	부산지면온도(°C) : 위와 같음  
traffic_mean : 일별 교통량 평균  
is_weekend : 주말인가에 대한 라벨링  
is_covid : 코로나 발발 이전이가 이후인가에 대한 라벨링  
date : 추석기준 날짜가 언제인지  
hour : 시간   

### 예측

In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings    # 경고메시지 제외
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.formula.api import ols
import scipy.stats as spst
warnings.filterwarnings(action='ignore')

plt.rc('font', family='NanumGothic')
sns.set(font="NanumGothic",
        rc={"axes.unicode_minus":False}, # 마이너스 부호 깨짐 현상 해결
        style='darkgrid')

df = pd.read_csv('./final_dataset.csv')

In [2]:
# 필요없는 column 제거
df.drop(columns=['Unnamed: 0', 'idx', 'year'], inplace=True, axis=1)
# 범주형 데이터 처리
df_d = pd.get_dummies(df, columns=['date', 'hour'])

In [3]:
df_d_time_df = df_d.copy()

In [4]:
df_d_time_df['DT'] = df_d_time_df['original_time']
df_d_time_df.set_index('DT', inplace=True)
df_d_time_df.tail()

,spend_time,original_time,서울기온(°C),서울강수량(mm),서울풍속(m/s),서울습도(%),서울이슬점온도(°C),서울일조(hr),서울일사(MJ/m2),서울시정(10m),...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
DT,,,,,,,,,,,,,,,,,,,,,
2021-09-24 20:00:00,301.1,2021-09-24 20:00:00,23.3,0.0,3.1,63,15.8,0.0,0.00,2000,...,0,0,0,0,0,0,1,0,0,0
2021-09-24 21:00:00,315.4,2021-09-24 21:00:00,22.4,0.0,3.0,66,15.7,0.0,0.00,2000,...,0,0,0,0,0,0,0,1,0,0
2021-09-24 22:00:00,329.7,2021-09-24 22:00:00,21.6,0.0,3.8,69,15.6,0.0,0.00,2000,...,0,0,0,0,0,0,0,0,1,0
2021-09-24 23:00:00,312.2,2021-09-24 23:00:00,20.9,0.0,3.3,70,15.2,0.0,0.00,2000,...,0,0,0,0,0,0,0,0,0,1
2022-09-10 12:00:00,0.0,2022-09-10 12:00:00,26.0,0.0,2.0,45,13.2,0.3,1.23,2000,...,0,0,0,0,0,0,0,0,0,0


In [5]:
target_df = df_d_time_df[840:]

In [9]:
target_df.drop(columns=['spend_time', 'original_time'], inplace=True)

In [6]:
df_d_time_df = df_d_time_df[:840]

In [7]:
from sklearn.model_selection import train_test_split

target = 'spend_time'
x = df_d_time_df.drop([target, 'original_time'], axis=1)
y = df_d_time_df.loc[:, target]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=42)

In [12]:
from sklearn.linear_model import Lasso
l1_model_2 = Lasso(alpha=0.1)
l1_model_2.fit(x, y)
l1_pred_2 = l1_model_2.predict(target_df)

In [13]:
l1_pred_2

array([343.3177293])

In [20]:
from sklearn.linear_model import Lasso
from sklearn.metrics import * 

l1_model_2 = Lasso(alpha=1)
l1_model_2.fit(x_train, y_train)
l1_pred_2 = l1_model_2.predict(x_test)

print('RMSE : ', round(mean_squared_error(y_test, l1_pred_2, squared=False), 5))
print('MAE  : ', round(mean_absolute_error(y_test, l1_pred_2), 5))
print("r2_score 점수 : {:.3f}".format(r2_score(y_test,l1_pred_2)))

RMSE :  34.17782
MAE  :  27.28419
r2_score 점수 : 0.572


In [22]:
from sklearn.linear_model import Ridge
l2_model = Ridge().fit(x_train, y_train)
l2_model.fit(x_train, y_train)
l2_pred = l2_model.predict(x_test)

print('RMSE : ', round(mean_squared_error(y_test, l2_pred, squared=False), 5))
print('MAE  : ', round(mean_absolute_error(y_test, l2_pred), 5))
print("r2_score 점수 : {:.3f}".format(r2_score(y_test, l2_pred)))

RMSE :  34.50092
MAE  :  28.34448
r2_score 점수 : 0.563


In [31]:
from sklearn.linear_model import ElasticNet
e_model = ElasticNet(alpha=0.1, l1_ratio=0.7)
e_model.fit(x_train, y_train)
e_pred = e_model.predict(x_test)

print('RMSE : ', round(mean_squared_error(y_test, e_pred, squared=False), 5))
print('MAE  : ', round(mean_absolute_error(y_test, e_pred), 5))
print("r2_score 점수 : {:.3f}".format(r2_score(y_test, e_pred)))

RMSE :  33.73674
MAE  :  27.14917
r2_score 점수 : 0.583


In [32]:
from sklearn.ensemble import GradientBoostingRegressor

gbr_model = GradientBoostingRegressor(random_state=42)
gbr_model.fit(x_train, y_train)

grb_pred = gbr_model.predict(x_test)

print('RMSE : ', round(mean_squared_error(y_test, grb_pred, squared=False), 5))
print('MAE  : ', round(mean_absolute_error(y_test, grb_pred), 5))
print("r2_score 점수 : {:.3f}".format(r2_score(y_test, grb_pred)))

RMSE :  26.36909
MAE  :  20.27554
r2_score 점수 : 0.745


In [34]:
import xgboost
xgb_model = xgboost.XGBRegressor()
xgb_model.fit(x_train, y_train)
xgb_pred = xgb_model.predict(x_test)

print('RMSE : ', round(mean_squared_error(y_test, xgb_pred, squared=False), 5))
print('MAE  : ', round(mean_absolute_error(y_test, xgb_pred), 5))
print("r2_score 점수 : {:.3f}".format(r2_score(y_test, xgb_pred)))

RMSE :  21.63585
MAE  :  15.278
r2_score 점수 : 0.828


In [51]:
import xgboost
xgb_model = xgboost.XGBRegressor()
xgb_model.fit(x, y)
xgb_pred = xgb_model.predict(target_df)
print(xgb_pred)

[322.58066]


In [52]:
322.58066 / 60

5.376344333333334

In [49]:
target_df['date_-1'] = 1
target_df['date_0'] = 0

In [50]:
pd.set_option('display.max_columns', None)
target_df

,서울기온(°C),서울강수량(mm),서울풍속(m/s),서울습도(%),서울이슬점온도(°C),서울일조(hr),서울일사(MJ/m2),서울시정(10m),서울지면온도(°C),부산기온(°C),부산강수량(mm),부산풍속(m/s),부산습도(%),부산이슬점온도(°C),부산일조(hr),부산일사(MJ/m2),부산시정(10m),부산지면온도(°C),traffic_mean,is_weekend,is_covid,date_-3,date_-2,date_-1,date_0,date_1,date_2,date_3,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
DT,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-09-10 12:00:00,26.0,0.0,2.0,45,13.2,0.3,1.23,2000,31.2,26.0,0.0,8.0,65,18.9,0.0,0.12,4000,30.5,12,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [42]:
import lightgbm as lgb
lgb_model = lgb.LGBMRegressor()
lgb_model.fit(x_train, y_train)
lgb_pred = lgb_model.predict(x_test)

print('RMSE : ', round(mean_squared_error(y_test, lgb_pred, squared=False), 5))
print('MAE  : ', round(mean_absolute_error(y_test, lgb_pred), 5))
print("r2_score 점수 : {:.3f}".format(r2_score(y_test, lgb_pred)))

RMSE :  22.46358
MAE  :  16.30563
r2_score 점수 : 0.815


In [44]:
from pycaret.regression import *

pc_model = setup(
  data = df_d_time_df,
  target = "spend_time",
  use_gpu = True,
  train_size=0.8,
  session_id = 42,
  fold=3
)

,Description,Value
0,session_id,42
1,Target,spend_time
2,Original Data,"(840, 54)"
3,Missing Values,False
4,Numeric Features,49
5,Categorical Features,3
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(672, 80)"


In [45]:
compare_models(sort="r2")

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,20.1855,886.8880,29.6867,0.7679,0.0876,0.0639,0.2900
xgboost,Extreme Gradient Boosting,20.1909,940.6240,30.6424,0.7530,0.0900,0.0646,0.3167
rf,Random Forest Regressor,20.1266,948.0453,30.7101,0.7510,0.0894,0.0636,0.5700
et,Extra Trees Regressor,18.5000,957.0234,30.9187,0.7483,0.0885,0.0584,0.5600
gbr,Gradient Boosting Regressor,22.2214,1009.0822,31.6849,0.7353,0.0938,0.0711,0.2067
ridge,Ridge Regression,25.7818,1178.8461,34.3012,0.6915,0.1062,0.0837,0.0133
br,Bayesian Ridge,25.8550,1179.0190,34.3086,0.6914,0.1059,0.0838,0.0600
lr,Linear Regression,25.9611,1209.1185,34.7210,0.6839,0.1081,0.0846,0.0267
ada,AdaBoost Regressor,29.0084,1321.8640,36.3124,0.6536,0.1133,0.0961,0.1533
lasso,Lasso Regression,31.3678,1748.9766,41.8006,0.5422,0.1245,0.0998,0.0433


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)